<div style="background-color:green; color:white">
<h1>Bike Index Project</h1>
</div>

<div style="background-color:black; color:white">

Author: **Mister Riley**  
- GitHub: [https://github.com/sorzkode/](https://github.com/sorzkode/)
- kaggle: [https://www.kaggle.com/code/misterriley/bikeindexeda](https://www.kaggle.com/code/misterriley/bikeindexeda)

<div style="background-color:green; color:white">
<h1>Extracting the data using the Bike Index API</h2>
</div>

In [1]:
# Import necessary libraries
import os
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# This will take approximately 30 minutes to run

# Function to get bike data from the API
def get_bike_data(api_key_name, api_key, page, per_page):
    # API URL
    api_url = "https://bikeindex.org/api/v3/search"
    
    # Include API key and key name in headers
    headers = {
        "API-Key-Name": api_key_name,
        "API-Key": api_key
    }

    # Specify parameters for pagination
    params = {
        "page": page,
        "per_page": per_page
    }

    # Make the API request
    response = requests.get(api_url, headers=headers, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        return response.json()  # Return the JSON response
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None

# Function to save data to a JSON file
def save_to_json(data):
    file_path = os.path.join("SourceData", "bikedata.json")

    # Check if the file already exists
    if os.path.exists(file_path):
        # Load existing data from the file
        with open(file_path, "r") as json_file:
            existing_data = json.load(json_file)
    else:
        existing_data = {"bikes": []}

    # Append the new bike data to the existing list under "bikes"
    existing_data["bikes"].extend(data["bikes"])

    # Write the updated data back to the file
    with open(file_path, "w") as json_file:
        json.dump(existing_data, json_file, indent=2)  # Write the data to the JSON file with indentation
        json_file.write('\n')  # Add a new line after each JSON object

    print(f"Data saved to {file_path}")

# Main function
def main():
    api_key_name = "bearer"
    api_key = "YOUR_API_KEY"
    per_page = 100

    # Loop through the pages (1 to 500)
    for page in range(1, 501):
        # Get bike data for the current page
        bike_data = get_bike_data(api_key_name, api_key, page, per_page)

        if bike_data:
            # Save the data to the combined JSON file
            save_to_json(bike_data)

# Check if the script is being run directly
if __name__ == "__main__":
    main()  # Call the main function to start the data extraction process

<div style="background-color:green; color:white">
<h1>Loading JSON data into a list</h2>
</div>

In [ ]:
# Define the folder path
folder_path = "SourceData"

# Get the list of JSON files in the folder
json_files = [file for file in os.listdir(folder_path) if file.endswith(".json")]

# Initialize an empty list to store the data
data_list = []

# Loop through each JSON file
for json_file in json_files:
    # Construct the file path
    file_path = os.path.join(folder_path, json_file)
    
    # Load JSON data
    print(f"Loading JSON data from {file_path}...")
    with open(file_path, 'r') as file:
        data = json.load(file)
    
    # Access the "bikes" key to get the list
    bikes_list = data.get("bikes", [])
    
    # Extend the data list with the bikes list
    data_list.extend(bikes_list)

print("JSON data loaded successfully!")

<div style="background-color:green; color:white">
<h1>Converting the list into a dataframe</h2>
</div>

In [ ]:
# Convert the data list to a DataFrame
df = pd.DataFrame(data_list)
print("DataFrame created successfully!")

# Print the shape of the DataFrame
print(f"Loaded {df.shape[0]} rows and {df.shape[1]} columns into the dataframe.")

<div style="background-color:green; color:white">
<h1>Exploring / transforming the data</h2>
</div>

In [ ]:
df.info() # Display the column names and data types

In [ ]:
df.head() # Display the first 5 rows of the DataFrame

In [ ]:
# Convert the date_stolen column to datetime format
try:
    df['date_stolen'] = pd.to_datetime(df['date_stolen'], errors='coerce', unit='s')
    print("date_stolen column converted to datetime format successfully!")
except Exception as e:
    print(f"Error occurred while converting the date_stolen column: {str(e)}")

# Validate the conversion
df['date_stolen'].head(10)

In [ ]:
earliest_date = df['date_stolen'].min(skipna=True) # Get the earliest date
latest_date = df['date_stolen'].max(skipna=True) # Get the latest date

print(f"Earliest date: {earliest_date}") # Print the earliest date
print(f"Latest date: {latest_date}") # Print the latest date

In [ ]:
df['month_stolen'] = pd.to_datetime(df['date_stolen']).dt.strftime('%B') # Extract the month from the date_stolen column
df['month_stolen'].unique() # Validate the new column

In [ ]:
# Convert the year column to datetime format
try: 
    df['year'] = pd.to_datetime(df['year'], format='%Y').dt.year
    print("year column converted to datetime format successfully!")
except Exception as e:
    print(f"Error occurred while converting the year column: {str(e)}")

# Validate the conversion
df['year'].unique()

In [ ]:
# Notice that the year column has at least 1 invalid value (2029). 
df[df['year'] > 2024] # Check for more invalid values

In [ ]:
# Define a function to correct the 'year' values
def correct_year(year):
    try:
        # Convert to integer and check if greater than the current year
        if int(year) > pd.Timestamp.now().year:
            return str(int(year) - 100)
        else:
            return str(year)
    except ValueError:
        # Handle non-numeric values (e.g., if 'year' is already a string)
        return year

# Apply the correction function to the 'year' column
df['year'] = df['year'].apply(correct_year)

# Print the updated DataFrame
df['year'].unique()


In [ ]:
# Extract latitude and longitude into separate columns
try:
    df[['latitude', 'longitude']] = df['stolen_coordinates'].apply(lambda x: pd.Series(x) if x else pd.Series([None, None]))
    print("Successfully split the 'stolen_coordinates' column into 'latitude' and 'longitude'.")
except Exception as e:
    print(f"Error occurred while splitting the 'stolen_coordinates' column: {str(e)}")

# Validate the split
df[['latitude', 'longitude']].head(10)

In [ ]:
# Check for missing values
print("\nMissing Values:")
df.isnull().sum() 

In [ ]:
# Interpolate missing values

df['year'].fillna(2020, inplace=True)
df['stolen_location'].fillna("Unknown", inplace=True)
df['date_stolen'].fillna("2020-01-01", inplace=True)
df['description'].fillna("Unknown", inplace=True)
df['frame_model'].fillna("Unknown", inplace=True)
df['stolen_coordinates'].fillna("Unknown", inplace=True)
df['latitude'].fillna("Unknown", inplace=True)
df['longitude'].fillna("Unknown", inplace=True)
df['month_stolen'].fillna("Unknown", inplace=True)

# Use boolean indexing for 'location_found' based on 'status'
if not 'stolen':
    df['location_found'].fillna(df['stolen_location'])
else:
    df['location_found'].fillna("Unknown", inplace=True)

# Validate the interpolation
df.isnull().sum()

In [ ]:
# Drop unnecessary columns
columns_to_drop = ['is_stock_img', 'large_img', 'external_id', 'registry_name', 'registry_url', 'thumb', 'url', 'propulsion_type_slug', 'cycle_type_slug', 'serial']

try:
    df.drop(columns=columns_to_drop, inplace=True) # Drop the columns from the DataFrame
    print("Columns dropped successfully!")
except Exception as e:
    print(f"Error occurred while dropping columns: {str(e)}")
    
# Validate the drop
df.info() 

In [ ]:
# Check unique values in categorical columns
print("\nUnique Values:")
for column in df.columns:
    if df[column].dtype == 'object': 
        unique_values = df[column].astype('str').nunique() 
        print(f"{column}: {unique_values} unique values") # Check unique values in categorical columns

In [ ]:
# Extract colors into separate columns
try:
    df_colors = df['frame_colors'].apply(lambda x: pd.Series(x) if x else pd.Series([None]*len(x)))
    # Rename columns to 'color1', 'color2', 'color3', etc.
    df_colors.columns = [f'color{i+1}' for i in range(df_colors.shape[1])]
    # Concatenate the new columns to the original DataFrame
    df = pd.concat([df, df_colors], axis=1)
    print("Successfully split the 'frame_colors' column")
except Exception as e:
    print(f"Error occurred while splitting the 'frame_colors' column: {str(e)}")

# Validate the split
df[['color1','color2','color3']].head(10)

In [ ]:
# Drop frame_colors column
try:
    df.drop(columns=['frame_colors'], inplace=True)
    print("Successfully dropped the 'frame_colors' column.")
except Exception as e:      
    print(f"Error occurred while dropping the 'frame_colors' column: {str(e)}")

# Validate the drop
df.info()

In [ ]:
# Check unique values in color columns
print('color1 unique values:', df['color1'].unique())
print('color2 unique values:', df['color2'].unique())
print('color3 unique values:', df['color3'].unique())

In [ ]:
# Cleaning the color columns
df['color1'] = df['color1'].replace('Silver, gray or bare metal', 'Silver')
df['color2'] = df['color2'].replace('Silver, gray or bare metal', 'Silver')
df['color3'] = df['color3'].replace('Silver, gray or bare metal', 'Silver')

df['color1'] = df['color1'].replace('Stickers tape or other cover-up', 'Decals')
df['color2'] = df['color2'].replace('Stickers tape or other cover-up', 'Decals')
df['color3'] = df['color3'].replace('Stickers tape or other cover-up', 'Decals')

# Filling missing values in color columns
df[['color2', 'color3']] = df[['color2', 'color3']].fillna('None')

# Validating the changes
print('color1 unique values:', df['color1'].unique())
print('color2 unique values:', df['color2'].unique())
print('color3 unique values:', df['color3'].unique())

In [ ]:
# Split manufacturer_name column
try:
    df[['manufacturer']] = df['manufacturer_name'].str.split(' ', expand=True)[[0]]
    print("Successfully split the 'manufacturer_name' column.")
except Exception as e:
    print(f"Error occurred while splitting the 'manufacturer_name' column: {str(e)}")

# Validate the split
df['manufacturer'].unique()

In [ ]:
location_columns = ['city', 'state', 'country']  # List of new column names
split_values = df['stolen_location'].str.split(',', expand=True).fillna('')  # Split the stolen_location column and fill missing values with empty string

for i, column in enumerate(location_columns):
    df[column] = split_values[i].str.strip()  # Remove leading and trailing whitespaces

df[location_columns].head(10)  # Validate the new columns

df.head(10)

In [ ]:
state_columns = ['state', 'zip']  # List of new column names
split_state = df['state'].str.split(' ', expand=True).fillna('')  # Split the stolen_location column and fill missing values with empty string

for i, column in enumerate(state_columns):
    df[column] = split_state[i].str.strip()  # Remove leading and trailing whitespaces

df[state_columns].head(10)  # Validate the new columns

In [ ]:
df.describe() # Display descriptive statistics for the dataframe

<div style="background-color:green; color:white">
<h1>Exporting data to perform some manual cleaning</h2>
</div>

In [ ]:
df.to_csv('SourceData/bikeindex.csv', index=False) # Save the cleaned data to a CSV file

<div style="background-color:green; color:white">
<h1>Importing Cleaned Data</h2>
</div>

In [ ]:
# Define the file path
file_path = "SourceData/bikeindex.csv"

# Import the CSV file into a dataframe with the correct encoding
df_cleaned = pd.read_csv(file_path, encoding='latin1')

# Display the dataframe
df_cleaned.head()


In [ ]:
df_cleaned.info() # Display basic information about the dataframe  

In [ ]:
df_cleaned.describe() # Display descriptive statistics for the dataframe


<div style="background-color:green; color:white">
<h1>Visualizing Cleaned Data</h2>
</div>

In [ ]:
# Count of stolen vs. not stolen bikes
plt.figure(figsize=(8, 5)) # Set figure size
df_cleaned['stolen'].value_counts().plot(kind='bar', color=['green', 'red']) # Plot count of stolen vs. not stolen bikes
plt.title('Count of Stolen vs. Not Stolen Bikes')
plt.xlabel('Stolen')
plt.ylabel('Count')
plt.show() # Display plot

In [ ]:
# Convert 'year' column to integer
df_cleaned['year'] = df_cleaned['year'].astype(float).astype(str)

# Subtract 3350 from the count of 'year' 2020
df_cleaned.loc[df_cleaned['year'] == 2020, 'year'] -= 3350

# Plot histogram of the "year" column
plt.figure(figsize=(10, 6))
top_10_years = df_cleaned['year'].value_counts().head(10)  # Get the top 10 most frequent years
plt.bar(top_10_years.index, top_10_years.values, color='skyblue')
plt.title('Frequency of Bike Thefts based on Model Year (Top 10)')
plt.xlabel('Year')
plt.ylabel('Frequency')
plt.xticks(rotation=45)  # Rotate x-axis labels for better visibility
plt.show()

In [ ]:
# Count the occurrences of each month
month_counts = df_cleaned[df_cleaned['month_stolen'] != 'Unknown']['month_stolen'].value_counts()

# Plot the bar chart
plt.figure(figsize=(8, 5))
top_10_months = month_counts.head(10)  # Get the top 10 most frequent months
top_10_months.plot(kind='bar', color='skyblue')
plt.title('Frequency of Bike Thefts by Month (Top 10)')
plt.xlabel('Month')
plt.ylabel('Number of Thefts')
plt.xticks(rotation=0)
plt.show()

In [ ]:
# Count of bikes by manufacturer (top 10)
top_10_manufacturers = df_cleaned['manufacturer'].value_counts().head(10)
plt.figure(figsize=(10, 6))
top_10_manufacturers.plot(kind='bar', color='skyblue')
plt.title('Frequency of Bike Thefts by Manufacturer (Top 10)')
plt.xlabel('Manufacturer')
plt.ylabel('Count')
plt.show()

In [ ]:
# Get the top 10 colors
top_10_colors = df_cleaned['color1'].value_counts().head(10)

# Plot the top 10 colors
plt.figure(figsize=(10, 6))
top_10_colors.plot(kind='bar', color='skyblue')
plt.title('Frequency of Bike Thefts by Color (Top 10)')
plt.xlabel('Color')
plt.ylabel('Count')
plt.show()

In [ ]:
df_cleaned['stolen_country'].unique() # Check unique values in 'stolen_country' column

In [ ]:
df_cleaned['stolen_country'] = df_cleaned['stolen_country'].replace({'CA': 'Canada', 'US': 'United States', 'IN': 'India', 'FR': 'France', 'ES': 'Spain', 'HU': 'Hungary', 'DE': 'Germany', 'GB': 'United Kingdom', 'AU': 'Australia', 'NL': 'Netherlands', 'PL': 'Poland', 'PT': 'Portugal', 'AT': 'Austria', 'RS': 'Serbia', 'CH': 'Switzerland', 'MX': 'Mexico', 'SG': 'Singapore', 'RO': 'Romania', 'DK': 'Denmark', 'CO': 'Columbia', 'BE': 'Belgium', 'PK': 'Pakistan', 'SK': 'Slovakia', 'IT': 'Italy', 'VN': 'Vietnam'}) # Replace country codes with country names

df_cleaned['country'].unique() # Validate

In [ ]:
df_cleaned['state'].head(10) # Display the 'state' column

In [ ]:
# Define a dictionary mapping US state abbreviations to their full names
us_states_mapping = {
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming'
}

# Replace 'stolen_country' column values with full names using the dictionary
df_cleaned['state'] = df_cleaned['state'].replace(us_states_mapping) 
df_cleaned['state'].head(50) # Validate


In [ ]:
df_cleaned['city'].head(50) # Display the 'city' column

In [ ]:
# Remove and interpolate missing values in 'city', 'state', and 'country' columns
df_cleaned['city'] = df_cleaned['city'].str.replace('city of', '') # Remove 'city of' from 'city' column values
df_cleaned['city'].fillna('Unknown', inplace=True) # Replace missing values with 'Unknown'
df_cleaned['state'].fillna('Unknown', inplace=True) # Replace missing values with 'Unknown'
df_cleaned['country'].fillna('Unknown', inplace=True) # Replace missing values with 'Unknown'

In [ ]:
df_cleaned.loc[df_cleaned['city'] == 'US', 'country'] = 'United States' # Replace 'US' in 'country' column with 'United States'
df_cleaned.head(10) # Validate

In [ ]:
# Count the occurrences of each country
country_counts = df_cleaned[df_cleaned['country'] != '']['country'].value_counts()

# Sort the countries in descending order
sorted_countries = country_counts.sort_values(ascending=False)

# Select the top 10 countries
top_10_countries = sorted_countries.head(10)

# Plot the bar chart
plt.figure(figsize=(12, 6))
top_10_countries.plot(kind='bar', color='skyblue')
plt.title('Top 10 Countries by Stolen Bikes')
plt.xlabel('Country')
plt.ylabel('Count')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Get the top 10 states
top_10_states = df_cleaned[(df_cleaned['country'] == 'United States') & (df_cleaned['state'] != '')]['state'].value_counts().head(10)

# Plot the bar chart
plt.figure(figsize=(10, 6))
top_10_states.plot(kind='bar', color='skyblue')
plt.title('Top 10 States in the United States by Bike Thefts')
plt.xlabel('State')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Get the top 10 cities
top_10_cities = df_cleaned[
    (df_cleaned['stolen_country'] == 'United States') 
    & 
    (df_cleaned['stolen_city'] != 'Unknown')]['stolen_city'].value_counts().head(10)

# Plot the bar chart
plt.figure(figsize=(10, 6))
top_10_cities.plot(kind='bar', color='skyblue')
plt.title('Top 10 Cities in the United States by Bike Thefts')
plt.xlabel('City')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Get the 10 states with the least thefts
bottom_10_states = df_cleaned[df_cleaned['stolen_country'] == 'United States']['stolen_state'].value_counts().tail(10)

# Plot the bar chart
plt.figure(figsize=(10, 6))
bottom_10_states.plot(kind='bar', color='skyblue')
plt.title('10 States with the Least Bike Thefts')
plt.xlabel('State')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Get the 10 cities with the least thefts
bottom_10_cities = df_cleaned[df_cleaned['stolen_country'] == 'United States']['stolen_city'].value_counts().tail(10)

# Plot the bar chart
plt.figure(figsize=(10, 6))
bottom_10_cities.plot(kind='bar', color='skyblue')
plt.title('10 Cities with the Least Bike Thefts')
plt.xlabel('State')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Save the cleaned data to a new CSV file
df_cleaned.to_csv('bikedata_cleaned.csv', index=False)